<a href="https://colab.research.google.com/github/sejongmin/AI/blob/main/%EC%8B%A4%EC%8A%B53_%EC%BB%A8%EB%B3%BC%EB%A3%A8%EC%85%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

import numpy as np

## MNIST 데이터셋 불러오기

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
mnist_train = datasets.MNIST(root='MNIST_data/',
                             train=True,
                             transform=transforms.ToTensor(),
                             download=True)

mnist_test = datasets.MNIST(root='MNIST_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 88428104.33it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 91423165.15it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 67149792.18it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 18459814.70it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [ ]:
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
mnist_train.data.shape

torch.Size([60000, 28, 28])

In [ ]:
input_dim = mnist_train.data.shape[1] * mnist_train.data.shape[2]
output_dim = 10

In [ ]:
train_loader = DataLoader(dataset=mnist_train, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=mnist_test, batch_size=100, shuffle=False)

## 다층 퍼셉트론으로 MNIST 손글씨 분류하기

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(input_dim,100)
        self.fc2 = nn.Linear(100,100)
        self.fc3 = nn.Linear(100,output_dim)

        self.relu = nn.ReLU()
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

MLP_classifier = MLP(input_dim, output_dim)

In [ ]:
criterion = nn.CrossEntropyLoss() # Logsoftmax(output을 확률로 변경해줌)가 포함되어 있음
optimizer = torch.optim.Adam(MLP_classifier.parameters(), lr=1e-2) # 최적화

In [ ]:
for epoch in range(10):
    for X, y in train_loader:
        optimizer.zero_grad()
        X = X.view(X.size(0), -1)
        y_pred = MLP_classifier(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

    print("loss of {} epoch: {}".format(epoch, loss.item()))

loss of 0 epoch: 0.044610075652599335
loss of 1 epoch: 0.16501951217651367
loss of 2 epoch: 0.08704157918691635
loss of 3 epoch: 0.053274668753147125
loss of 4 epoch: 0.13130001723766327
loss of 5 epoch: 0.024165237322449684
loss of 6 epoch: 0.3589969575405121
loss of 7 epoch: 0.05899108946323395
loss of 8 epoch: 0.1964663565158844
loss of 9 epoch: 0.02736964263021946


In [ ]:
tmp = torch.randn((100, 1, 28, 28))
tmp.shape

torch.Size([100, 1, 28, 28])

In [ ]:
tmp = tmp.view(tmp.size(0), -1)
tmp.shape

torch.Size([100, 784])

In [ ]:
MLP_classifier.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for X_test, y in test_loader:
        X_test = X_test.view(X_test.size(0), -1)
        y_pred = MLP_classifier(X_test)
        _, predicted = torch.max(y_pred.data, 1)
        correct += predicted.eq(y.data.view_as(predicted)).sum()

    # 정확도 출력
    data_num = len(test_loader.dataset)  # 데이터 총 건수
    print('\n테스트 데이터 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))


테스트 데이터 예측 정확도: 9650/10000 (96%)



## CNN으로 MNIST 손글씨 분류하기

- 1번 레이어 : 합성곱층(Convolutional layer)
 - 합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
 - 맥스풀링(kernel_size=2, stride=2))

- 2번 레이어 : 합성곱층(Convolutional layer)
 - 합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
 - 맥스풀링(kernel_size=2, stride=2))

- 3번 레이어 : 전결합층(Fully-Connected layer)
 - 전결합층 (입력 **????** / 출력 10)

In [ ]:
tmp = torch.randn((100, 1, 28, 28))
conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

In [ ]:
tmp = conv1(tmp)
tmp.shape

torch.Size([100, 32, 28, 28])

In [ ]:
tmp = maxpool(tmp)
tmp.shape

torch.Size([100, 32, 14, 14])

In [ ]:
tmp = conv2(tmp)
tmp.shape

torch.Size([100, 64, 14, 14])

In [ ]:
tmp = maxpool(tmp)
tmp.shape

torch.Size([100, 64, 7, 7])

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc = torch.nn.Linear(7 * 7 * 64, 10)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

CNN_classifier = CNN()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNN_classifier.parameters(), lr=1e-2)

In [ ]:
for epoch in range(3):
    for X, y in train_loader:
        optimizer.zero_grad()
        y_pred = CNN_classifier(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

    print("loss of {} epoch: {}".format(epoch, loss.item()))

loss of 0 epoch: 0.14416253566741943
loss of 1 epoch: 0.10696843266487122
loss of 2 epoch: 0.03293764591217041


In [ ]:
CNN_classifier.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for X_test, y in test_loader:
        y_pred = CNN_classifier(X_test)
        _, predicted = torch.max(y_pred.data, 1) #뒤에 1은 차원을 맞춤
        correct += predicted.eq(y.data.view_as(predicted)).sum()

    # 정확도 출력
    data_num = len(test_loader.dataset)  # 데이터 총 건수
    print('\n테스트 데이터 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))


테스트 데이터 예측 정확도: 9774/10000 (98%)

